# KRR Project




## Team Members: Isaam Ansari, Mohammad Isaam, Isaam Ul Haq
## Roll Number: 21I-0299, 21I-0299, 21I-0299

Importing Libraries

In [1295]:
import pandas as pd
from owlready2 import *
from rdflib import Graph
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.graph import Collection
from rdflib import Namespace, BNode
import rdflib
from unidecode import unidecode
from datetime import datetime
import xmlschema



In [1296]:
#Load data from CSV and Excel files
athlete_data = pd.read_csv(r"C:\Users\USER\Olympics\athletes.csv")
coach_data = pd.read_csv(r"C:\Users\USER\Olympics\coaches.csv")
medals_total_data = pd.read_csv(r"C:\Users\USER\Olympics\medals_total.csv")
medals_data = pd.read_csv(r"C:\Users\USER\Olympics\medals.csv")
officials_data = pd.read_csv(r"C:\Users\USER\Olympics\technical_officials.csv")
coaches_xlsx = pd.read_excel(r"C:\Users\USER\Olympics\Coaches.xlsx")
entries_gender_xlsx = pd.read_excel(r"C:\Users\USER\Olympics\EntriesGender.xlsx")
teams_xlsx = pd.read_excel(r"C:\Users\USER\Olympics\Teams.xlsx")
athlete_data['discipline'] = athlete_data.groupby('discipline_code')['discipline'].transform(lambda x: x.fillna(x.mode().iloc[0]))
entiredf = pd.read_csv(r"C:\Users\USER\Olympics\2020_Olympics_Dataset.csv", encoding='ISO-8859-1', index_col=0)
athlete_data.loc[athlete_data['country']=='Samoa (until 1996 Western Samoa)','country']='Samoa'
#countries = pd.read_csv(r"C:\Users\USER\Olympics\countries_codes_and_coordinates.csv")
#countries['Alpha-3 code'] = countries['Alpha-3 code'].apply(lambda x: x[2:-1])
#athlete_data.loc[country]=='Samoa'
#countries.loc[countries['Country'] == 'Germany', 'Alpha-3 code'] = 'GER'

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [1297]:
athevents = {}

for i in entiredf['Sport'].unique().tolist():
    athevents[i]=[]

for i,r in entiredf.iterrows():
    #print(r['Event'])
    if(r['Event'] not in athevents[r['Sport']]):
        athevents[r['Sport']].append(r['Event'])
        
print(athevents)

{'Cycling Road': ["Women's Road Race", "Women's Individual Time Trial", "Men's Road Race", "Men's Individual Time Trial"], 'Artistic Gymnastics': ["Men's All-Around", "Men's Floor Exercise", "Men's Pommel Horse", "Men's Rings", "Men's Vault", "Men's Parallel Bars", "Men's Horizontal Bar", "Men's Team", "Women's All-Around", "Women's Vault", "Women's Uneven Bars", "Women's Balance Beam", "Women's Floor Exercise", "Women's Team"], 'Rowing': ["Men's Pair Team", "Lightweight Men's Double Sculls Team", "Women's Single Sculls", "Men's Eight Team", "Men's Double Sculls Team", "Women's Quadruple Sculls Team", "Women's Pair Team", "Men's Single Sculls", "Women's Eight Team", "Lightweight Women's Double Sculls Team", "Women's Four Team", "Men's Quadruple Sculls Team", "Women's Double Sculls Team", "Men's Four Team"], 'Basketball': ['Men Team', 'Women Team'], 'Handball': ['Men Team', 'Women Team'], 'Swimming': ["Men's 100m Breaststroke", "Men's 100m Freestyle", "Men's 1500m Freestyle", 'Mixed 4 x

Things to convert to triples:


-Create each athlete.

Each athlete's height, weight and nationality

-Create each coach.

Each coach coaches an athlete.

Each athlete is coached by a coach.

-Create each country

Each athlete represents a single country

Each country has a rank

-Create each event & discipline

Athletes participate in each event

Events have a discipline.


In [1298]:
nocs = entiredf['NOC'].unique().tolist()
countries = entiredf['Country'].unique().tolist()
names = entiredf['Name'].unique().tolist()
namegenders = []
namecountries = []
for i in range(len(names)):
    namegenders.append(entiredf[entiredf['Name']==names[i]]['Gender'].unique()[0])
    namecountries.append(entiredf[entiredf['Name']==names[i]]['Country'].unique()[0])

In [1299]:
print(names[:5])
print(namegenders[:5])
print(namecountries[:5])
print('<------------------------------------> Countries <----------------------------------------------->')
print(countries[:5])
print(nocs[:5])

['AALERUD Katrine', 'ABAD Nestor', 'ABAGNALE Giovanni', 'ABALDE Alberto', 'ABALDE Tamara']
['Female', 'Male', 'Male', 'Male', 'Female']
['Norway', 'Spain', 'Italy', 'Spain', 'Spain']
<------------------------------------> Countries <----------------------------------------------->
['Norway', 'Spain', 'Italy', 'France', 'Chile']
['NOR', 'ESP', 'ITA', 'FRA', 'CHI']


In [1300]:
#len(entiredf['Name'].unique())
#for i in range(len(names)):
#    if(len(entiredf[entiredf['Name']==names[i]]['Discipline'].unique())>1):
#        print(entiredf[entiredf['Name']==names[i]])

In [1301]:
entiredf

,Code,Name,Gender,Age,NOC,Country,Discipline,Sport,Event,Rank,Medal
1,1346266,AALERUD Katrine,Female,26,NOR,Norway,CRD,Cycling Road,Women's Road Race,37.0,NaN
2,1346266,AALERUD Katrine,Female,26,NOR,Norway,CRD,Cycling Road,Women's Individual Time Trial,20.0,NaN
3,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's All-Around,NaN,NaN
4,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's Floor Exercise,NaN,NaN
5,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's Pommel Horse,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15117,1317127,ZWOLINSKA Klaudia,Female,22,POL,Poland,CSL,Canoe Slalom,Women's Kayak,5.0,NaN
15118,1428770,ZYKOVA Yulia,Female,25,ROC,Russia,SHO,Shooting,50m Rifle 3 Positions Women,2.0,Silver
15119,1435606,ZYUZINA Ekaterina,Female,24,ROC,Russia,SAL,Sailing,Women's One Person Dinghy - Laser Radial,27.0,NaN
15120,1325697,ZYZANSKA Sylwia,Female,24,POL,Poland,ARC,Archery,Women's Individual,33.0,NaN


In [1302]:
coachnames = coach_data['name'].unique().tolist()
len(coachnames)
coachgenders = []
coachcountries = []
coachroles = []
coachevents = []
coachurls = []
coachbdays = []
for i in range(len(coachnames)):
    coachgenders.append(coach_data[coach_data['name']==coachnames[i]]['gender'].unique()[0])
    coachcountries.append(coach_data[coach_data['name']==coachnames[i]]['country_code'].unique()[0])
    coachroles.append(coach_data[coach_data['name']==coachnames[i]]['function'].unique()[0])
    ########### CHANGE AND FIX: coachevents.append(coach_data[coach_data['name']==coachnames[i]]['event'].unique()[0])
    coachbdays.append(coach_data[coach_data['name']==coachnames[i]]['birth_date'].unique()[0])
    coachurls.append(coach_data[coach_data['name']==coachnames[i]]['url'].unique()[0])

In [1303]:
for i in range(len(coachnames)):
    ce = coach_data[coach_data['name']==coachnames[i]]['event'].unique()
    if(len(ce)>1):
        for j in ce:
            print(ce)

['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Men' 'Women']
['Men' 'Women']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']
['Duet' 'Team']


In [1304]:
coach_data

,name,short_name,gender,birth_date,country_code,discipline,function,event,url
0,ABDELMAGID Wael,ABDELMAGID W,Male,1982-08-02,EGY,Football,Head Coach,NaN,../../../en/results/football/athlete-profile-n...
1,ABE Junya,ABE J,Male,1990-07-25,JPN,Volleyball,Head Coach,NaN,../../../en/results/volleyball/athlete-profile...
2,ABE Katsuhiko,ABE K,Male,1979-09-23,JPN,Basketball,Coach,NaN,../../../en/results/basketball/athlete-profile...
3,ADAMA Cherif,ADAMA C,Male,1962-05-06,CIV,Football,Head Coach,NaN,../../../en/results/football/athlete-profile-n...
4,AGEBA Yuya,AGEBA Y,Male,1983-09-30,JPN,Volleyball,Head Coach,NaN,../../../en/results/volleyball/athlete-profile...
...,...,...,...,...,...,...,...,...,...
402,ZAMORA PEDREIRA Javier,ZAMORA PEDREIRA J,Male,1984-10-27,ESP,Basketball,Coach,NaN,../../../en/results/basketball/athlete-profile...
403,ZAMPIERI Francesca,ZAMPIERI F,Female,1981-12-22,LIE,Artistic Swimming,Coach,Duet,../../../en/results/artistic-swimming/athlete-...
404,ZHANG Xiaohuan,ZHANG X,Female,1980-08-19,CHN,Artistic Swimming,Coach,NaN,../../../en/results/artistic-swimming/athlete-...
405,ZIJP Simon,ZIJP S,Male,1964-03-12,NED,Hockey,Coach,NaN,../../../en/results/hockey/athlete-profile-n14...


In [1305]:
entries_gender_xlsx

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041
5,Badminton,86,87,173
6,Baseball/Softball,90,144,234
7,Basketball,144,144,288
8,Beach Volleyball,48,48,96
9,Boxing,102,187,289


In [1306]:
teamnames = teams_xlsx['Name'].unique().tolist()
teamnocs = teams_xlsx['NOC'].unique().tolist()
medals_data
#teamnocs

,medal_type,medal_code,medal_date,athlete_short_name,athlete_name,athlete_sex,athlete_link,country_code,discipline_code,event,country,discipline
0,Gold Medal,1,2021-07-24 00:00:00.0,KIM JD,KIM Je Deok,X,../../../en/results/archery/athlete-profile-n1...,KOR,ARC,Mixed Team,Republic of Korea,Archery
1,Gold Medal,1,2021-07-24 00:00:00.0,AN S,AN San,X,../../../en/results/archery/athlete-profile-n1...,KOR,ARC,Mixed Team,Republic of Korea,Archery
2,Silver Medal,2,2021-07-24 00:00:00.0,SCHLOESSER G,SCHLOESSER Gabriela,X,../../../en/results/archery/athlete-profile-n1...,NED,ARC,Mixed Team,Netherlands,Archery
3,Silver Medal,2,2021-07-24 00:00:00.0,WIJLER S,WIJLER Steve,X,../../../en/results/archery/athlete-profile-n1...,NED,ARC,Mixed Team,Netherlands,Archery
4,Bronze Medal,3,2021-07-24 00:00:00.0,ALVAREZ L,ALVAREZ Luis,X,../../../en/results/archery/athlete-profile-n1...,MEX,ARC,Mixed Team,Mexico,Archery
...,...,...,...,...,...,...,...,...,...,...,...,...
2396,Bronze Medal,3,2021-08-08 00:00:00.0,NAGY V,NAGY Viktor,M,../../../en/results/water-polo/athlete-profile...,HUN,WPO,Men,Hungary,Water Polo
2397,Bronze Medal,3,2021-08-08 00:00:00.0,VAMOS M,VAMOS Marton,M,../../../en/results/water-polo/athlete-profile...,HUN,WPO,Men,Hungary,Water Polo
2398,Bronze Medal,3,2021-08-08 00:00:00.0,VARGA D,VARGA Denes,M,../../../en/results/water-polo/athlete-profile...,HUN,WPO,Men,Hungary,Water Polo
2399,Bronze Medal,3,2021-08-08 00:00:00.0,VOGEL S,VOGEL Soma,M,../../../en/results/water-polo/athlete-profile...,HUN,WPO,Men,Hungary,Water Polo


# Graph Creation - Run From Here

In [1320]:
# Define the namespaces
dc = Namespace("http://purl.org/dc/elements/1.1/")
dbo = Namespace("http://dbpedia.org/ontology/")
dbp = Namespace("http://dbpedia.org/property/")
dbr = Namespace("http://dbpedia.org/resource/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
xml = Namespace("httw://www.w3.org/XML/1998/namespace")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
schema = Namespace("http://schema.org/")
oly =  Namespace("http://olympicslinked.org/")

try:
    g.destroy(all)
except:
    None

g = Graph()
g.parse(r'C:/USERS/User/Downloads/KRR-KGLD-Fall2023-J06-Ontology.owl')
print(g.serialize(format="turtle"))


@prefix : <http://olympicslinked.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

: a owl:Ontology .

:AthleteWithAtleastOneMedal a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( :Athlete [ a owl:Restriction ;
                        owl:onProperty :awardedMedal ;
                        owl:someValuesFrom [ a owl:Class ;
                                owl:unionOf ( :BronzeMedal :GoldMedal :SilverMedal ) ] ] ) ] .

:HighPerformanceTeam a owl:Class ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:onProperty :hasRank ;
            owl:someValuesFrom [ a rdfs:Datatype ;
                    owl:onDatatype xsd:integer ;
                    owl:withRestrictions ( [ xsd:minExclusive 10 ] ) ] ] .

:NonCompetingAthlete a owl

In [1321]:
def create_uri(name, class_type):
    # Replace spaces and special characters in the name to create a URI-friendly string
    uri_name = name.replace(" ", "_").replace("'", "").replace(".", "").replace(",","")
    uri_name = unidecode(uri_name)
    if(class_type == ''):
        return URIRef(oly[f"{uri_name}"])
    else:
        return URIRef(oly[f"{class_type}/{uri_name}"])


def add_attribute(graph, instance_uri, attribute_uri, attribute_value):
    try:
        attribute_value = unidecode(attribute_value)
    except:
        None
    attribute_uri = unidecode(attribute_uri)
    graph.add((instance_uri, attribute_uri, Literal(attribute_value)))

# Events & Disciplines

# Countries

In [1322]:
# Iterate through each row in the DataFrame
for index in range(len(countries)):    
    c = countries[index]
    c = c.replace(" ", "_").replace("'", "").replace(",", "").replace("Saint","St.")
    c = unidecode(c)
    country_uri = URIRef(f"http://dbpedia.org/resource/{c}")
    #country_uri = create_uri(countries[index], 'Resource')
    print(country_uri)
    g.add((country_uri, RDF.type, oly.Country))
    
    try:
        #print(medals_total_data[medals_total_data['Country Code']==nocs[index]]['Rank'].unique().tolist()[0])
        add_attribute(g, country_uri, oly.hasRank, medals_total_data[medals_total_data['Country Code']==nocs[index]]['Rank'].unique().tolist()[0])
    except:
        None
    # Add other attributes using the add_attribute function
    add_attribute(g, country_uri, FOAF.name, countries[index])
    add_attribute(g, country_uri, RDFS.label, nocs[index])

http://dbpedia.org/resource/Norway
http://dbpedia.org/resource/Spain
http://dbpedia.org/resource/Italy
http://dbpedia.org/resource/France
http://dbpedia.org/resource/Chile
http://dbpedia.org/resource/Sudan
http://dbpedia.org/resource/Iran
http://dbpedia.org/resource/Azerbaijan
http://dbpedia.org/resource/Netherlands
http://dbpedia.org/resource/Australia
http://dbpedia.org/resource/United_States
http://dbpedia.org/resource/Qatar
http://dbpedia.org/resource/Egypt
http://dbpedia.org/resource/Belgium
http://dbpedia.org/resource/Malaysia
http://dbpedia.org/resource/Singapore
http://dbpedia.org/resource/Maldives
http://dbpedia.org/resource/Saudi_Arabia
http://dbpedia.org/resource/Germany
http://dbpedia.org/resource/Uzbekistan
http://dbpedia.org/resource/Indonesia
http://dbpedia.org/resource/Kazakhstan
http://dbpedia.org/resource/Bahrain
http://dbpedia.org/resource/Japan
http://dbpedia.org/resource/Ethiopia
http://dbpedia.org/resource/Canada
http://dbpedia.org/resource/Malta
http://dbpedia.or

# Athletes, Events, Medals & Coaches

In [1323]:
# Iterate through each row in the DataFrame
athcodes = []
events = {}
g.add((oly.Athlete, RDF.type, owl.Class))
g.add((oly.Country, RDF.type, owl.Class))
gold = 1
bronze = 1
silver = 1
for index, row in athlete_data[:].iterrows():
    # Create a new URI for each athlete using the helper function
    
    athlete_uri = create_uri(str(index), 'Athlete')
    #print(athlete_uri)
    c = row['country']
    c = c.replace(" ", "_").replace("'", "").replace(",", "").replace("Saint","St.")
    c = unidecode(c)
    country_uri = URIRef(f"http://dbpedia.org/resource/{c}")
    #print(country_uri)
    # Add RDF type for Athlete
    g.add((athlete_uri, RDF.type, oly.Athlete))
    
    #g.add((country_uri, oly.hasMember, athlete_uri)) -> inferred by reasoner
    g.add((athlete_uri, oly.partOfTeam, country_uri))
    
    #add_attribute(g, athlete_uri, oly.partOfTeam, country_uri)
    
    # Add other attributes using the add_attribute function
    add_attribute(g, athlete_uri, FOAF.name, row['name'])
    add_attribute(g, athlete_uri, RDFS.label, row['name'])
    add_attribute(g, athlete_uri, FOAF.gender, row['gender'])
    add_attribute(g, athlete_uri, FOAF.birthday, row['birth_date'])
    if(type(row['birth_place'])!=float):
        add_attribute(g, athlete_uri, oly.birthPlace, row['birth_place'])

    add_attribute(g, athlete_uri, oly.hasNationality, row['country'])
    
    if(type(row['birth_date'])!=float):
        #print(row['birth_date'])
        today = datetime.today()
        birth_date = datetime.strptime(row['birth_date'], '%Y-%m-%d')
        age = int(today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day)))
        #xsd_age = xmlschema.XMLSchema.to_python(age, 'integer')
        #print(athlete_uri)
        #g.add((athlete_uri, oly.hasAge, age))
        add_attribute(g, athlete_uri, oly.hasAge , int(age))
        #print(age)
    else:
        #g.add((athlete_uri, oly.hasAge, 0))
        add_attribute(g, athlete_uri, oly.hasAge , 0)
    #athlete participated in what event 
    #& won what medal if any (this is to be done in medal linking part)    
    
    prf = row['url'].index('athlete-profile-n')
    athcode = int(row['url'][prf+17:prf+24])
    if(athcode not in athcodes):
        athcodes.append(athcode)
        #print(athcodes)
        for i,r in entiredf[entiredf['Code']==athcode].iterrows():
            #print(r)
            duri = create_uri(r['Sport'],'Discipline')
            #print(duri)
            try:
                len(events[r['Sport']])
            except:
                events[r['Sport']]=[]
                g.add((duri, RDF.type, oly.Discipline))
                add_attribute(g, duri, RDFS.label, r['Discipline'])
                add_attribute(g, duri, FOAF.name, r['Sport'])
            urii = r['Sport'].replace(" ", "_").replace("'", "").replace(".", "")
            urii = unidecode(urii)
            eventuri = create_uri(r['Event'], f'Discipline/{urii}')  
            #print(eventuri)
            try:
                #Check if events[r['Sport']] has the event you're looking for already
                events[r['Sport']].index(r['Event'])
            except:
                events[r['Sport']].append(r['Event'])    
                #print(eventuri)
                g.add((eventuri, RDF.type, oly.Event))
                
                g.add((eventuri, oly.partOfDiscipline, duri))
                #infers the other way round
                add_attribute(g, eventuri, FOAF.name,(r['Sport']+" "+r['Event']))
                add_attribute(g, eventuri, RDFS.label,unidecode(r['Sport']+" "+r['Event']).replace("'",""))
            
            #print(r['Sport'],r['Event'],events[r['Sport']])
            g.add((athlete_uri, oly.competesIn, eventuri))
            if(type(r['Medal'])==float):
                print('Participation')   
            elif(r['Medal']=='Gold'):
                #print('Gold')
                #create gold medal uri new and then medal is medalOf eventuri, medal medalType not necessary i thinks
                medaluri = create_uri(str(gold), 'GoldMedal')
                g.add((medaluri, RDF.type, oly.GoldMedal))
                g.add((medaluri, oly.medalOfEvent, eventuri))
                g.add((athlete_uri, oly.awardedMedal, medaluri))
                add_attribute(g, medaluri, RDFS.label,("Gold Medal "+r['Sport']+" "+r['Event']).replace("'",""))
                #print(medaluri)
                gold+=1
            elif(r['Medal']=='Bronze'):
                medaluri = create_uri(str(bronze), 'BronzeMedal')
                g.add((medaluri, RDF.type, oly.BronzeMedal))
                g.add((medaluri, oly.medalOfEvent, eventuri))
                g.add((athlete_uri, oly.awardedMedal, medaluri))
                add_attribute(g, medaluri, RDFS.label,("Bronze Medal "+r['Sport']+" "+r['Event']).replace("'",""))
                #print(medaluri)                
                bronze+=1
                #print('Bronze')
            elif(r['Medal']=='Silver'):
                medaluri = create_uri(str(silver), 'SilverMedal')
                g.add((medaluri, RDF.type, oly.SilverMedal))
                g.add((medaluri, oly.medalOfEvent, eventuri))
                g.add((athlete_uri, oly.awardedMedal, medaluri))
                add_attribute(g, medaluri, RDFS.label,("Silver Medal "+r['Sport']+" "+r['Event']).replace("'",""))
                print(medaluri)                
                silver+=1
                #print('Silver')
        #print("Hi")        
                
        
    #print(len(entiredf[entiredf['Code']==athcode]['Event']))
    
    # Serialize the graph in RDF/XML format
    #print(g.serialize(format='turtle'))

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/1
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/2
Participation
http://olympicslinked.org/SilverMedal/3
Participation
Participation
Participatio

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/27
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/28
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/29
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/30
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/31
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participati

Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/61
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/62
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/63
Participation
Participation
http://olympicslinked.org/SilverMedal/64
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/65
Participation
http://olympicslinked.org/SilverMedal/66
Participation
Participatio

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/95
http://olympicslinked.org/SilverMedal/96
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMeda

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/127
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/128
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/129
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Partic

...

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/652
Participation
http://olympicslinked.org/SilverMedal/653
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Partic

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/681
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/682
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/683
Participation
Participation
http://olympicslinked.org/SilverMedal/684
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Partic

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/710
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/711
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/712
Participation
Participation
Participation
Partic

Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/749
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Partic

Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/779
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/780
http://olympicslinked.org/SilverMedal/781
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
Participation
http://olympicslinked.org/SilverMedal/782
Participation
Participation
Participation


In [1324]:
# Serialize the graph in RDF/XML format
#print(g.serialize(format='turtle'))

In [1325]:
coach_data
#coaches_xlsx

,name,short_name,gender,birth_date,country_code,discipline,function,event,url
0,ABDELMAGID Wael,ABDELMAGID W,Male,1982-08-02,EGY,Football,Head Coach,NaN,../../../en/results/football/athlete-profile-n...
1,ABE Junya,ABE J,Male,1990-07-25,JPN,Volleyball,Head Coach,NaN,../../../en/results/volleyball/athlete-profile...
2,ABE Katsuhiko,ABE K,Male,1979-09-23,JPN,Basketball,Coach,NaN,../../../en/results/basketball/athlete-profile...
3,ADAMA Cherif,ADAMA C,Male,1962-05-06,CIV,Football,Head Coach,NaN,../../../en/results/football/athlete-profile-n...
4,AGEBA Yuya,AGEBA Y,Male,1983-09-30,JPN,Volleyball,Head Coach,NaN,../../../en/results/volleyball/athlete-profile...
...,...,...,...,...,...,...,...,...,...
402,ZAMORA PEDREIRA Javier,ZAMORA PEDREIRA J,Male,1984-10-27,ESP,Basketball,Coach,NaN,../../../en/results/basketball/athlete-profile...
403,ZAMPIERI Francesca,ZAMPIERI F,Female,1981-12-22,LIE,Artistic Swimming,Coach,Duet,../../../en/results/artistic-swimming/athlete-...
404,ZHANG Xiaohuan,ZHANG X,Female,1980-08-19,CHN,Artistic Swimming,Coach,NaN,../../../en/results/artistic-swimming/athlete-...
405,ZIJP Simon,ZIJP S,Male,1964-03-12,NED,Hockey,Coach,NaN,../../../en/results/hockey/athlete-profile-n14...


# Coaches

In [1326]:
#Coach URIs

#Coach URI -> a Coach DONE
#Coach URI rdfs:label & foaf:name DONE
#Coach -> partOfTeam country URI DONE
#Coach -> foaf:birthday birthdate
#Coach -> foaf:gender gender DONE
#Coach -> coaches -> Discipline

for index,row in coach_data.iterrows():
    coachuri = create_uri(str(index+1),'Coach')
    g.add((coachuri,RDF.type,oly.Coach))
    #print(row['country_code'])
    #print(athlete_data[athlete_data['country_code']==(row['country_code'])]['country'].unique().tolist())
    #print(countries['Alpha-3 code'][0])
    c = athlete_data[athlete_data['country_code']==(row['country_code'])]['country'].unique().tolist()[0]
    c = c.replace(" ", "_").replace("'", "").replace(",", "").replace("Saint","St.")
    c = unidecode(c)
    country_uri = URIRef(f"http://dbpedia.org/resource/{c}")
    g.add((coachuri,oly.partOfTeam,country_uri))
    add_attribute(g,coachuri,FOAF.name,row['name'])
    add_attribute(g,coachuri,RDFS.label,row['name'])
    add_attribute(g,coachuri,FOAF.gender,row['gender'])
    bday = row['birth_date']
    add_attribute(g,coachuri,FOAF.birthday,bday)
    duri = create_uri(row['discipline'],'Discipline')
    g.add((coachuri,oly.coaches,duri))
    #print(coachuri,country_uri,duri)

In [1327]:
len(coach_data)

407

# Technical Officials

In [1328]:
for index,row in officials_data.iterrows():
    officialuri = create_uri(str(index+1),'TechnicalOfficial')
    g.add((officialuri,RDF.type,oly.TechnicalOfficial))
    
    #print(row['country_code'])
    #print(athlete_data[athlete_data['country_code']==(row['country_code'])]['country'].unique().tolist())
    #print(countries['Alpha-3 code'][0])
    
    c = row['country']
    c = c.replace(" ", "_").replace("'", "").replace(",", "").replace("Saint","St.")
    c = unidecode(c)
    country_uri = URIRef(f"http://dbpedia.org/resource/{c}")
    
    #g.add((officialuri,oly.,country_uri))
    
    add_attribute(g,officialuri,oly.hasNationality,row['country'])
    add_attribute(g,officialuri,FOAF.name,row['name'])
    add_attribute(g,officialuri,RDFS.label,row['name'])
    add_attribute(g,officialuri,FOAF.gender,row['gender'])
    bday = row['birth_date']
    add_attribute(g,officialuri,FOAF.birthday,bday)
    duri = create_uri(row['discipline'],'Discipline')
    g.add((officialuri,oly.officialOf,duri))
    
    #print(officialuri)
#    duri = create_uri(row['discipline'],'Discipline')
#    g.add((coachuri,oly.coaches,duri))

In [1329]:
officials_data

,name,short_name,gender,birth_date,country,discipline,function,url
0,ABAEVA Elena,ABAEVA E,Female,1966-04-21,Uzbekistan,Wrestling,Judge,../../../en/results/wrestling/athlete-profile-...
1,ABBAR Bachir,ABBAR B,Male,1965-05-03,Morocco,Boxing,Judge,../../../en/results/boxing/athlete-profile-n15...
2,ABDELLATIF Makfouni,ABDELLATIF M,Male,1972-11-23,Morocco,Boxing,Judge,../../../en/results/boxing/athlete-profile-n15...
3,ABE Miya,ABE M,Female,1992-10-27,Japan,Beach Volleyball,Referee,../../../en/results/beach-volleyball/athlete-p...
4,ACIGA FULA Antonio Stephen,ACIGA FULA AS,Male,1957-11-28,Uganda,Boxing,Judge,../../../en/results/boxing/athlete-profile-n15...
...,...,...,...,...,...,...,...,...
951,ZINGER Edward,ZINGER E,Male,1969-07-21,Canada,Wrestling,Judge,../../../en/results/wrestling/athlete-profile-...
952,ZOURAK Adil,ZOURAK A,Male,1978-08-25,Morocco,Football,Video Assistant Referee (VAR),../../../en/results/football/athlete-profile-n...
953,ZWART Michiel,ZWART M,Male,1986-11-08,Netherlands,Water Polo,Referee,../../../en/results/water-polo/athlete-profile...
954,ZWI Noam,ZWI N,Male,1958-04-20,Israel,Diving,Judge,../../../en/results/diving/athlete-profile-n15...


# Export Populated Owl File

In [1330]:
owl_file_path = 'C:/USERS/User/Downloads/pop.owl'

# Serialize the graph in RDF/XML format and save it
with open(owl_file_path, 'w') as owl_file:  # Using 'w' for text-based format
    owl_file.write(g.serialize(format='pretty-xml'))

print(f"OWL file saved at {owl_file_path}")


C:\ProgramData\Anaconda3\lib\site-packages\rdflib\plugins\serializers\rdfxml.py:280: UserWarning: Assertions on rdflib.term.BNode('Ne7eae22d57674d6787eee55aad340d56') other than RDF.first and RDF.rest are ignored ... including RDF.List
  self.predicate(predicate, object, depth + 1)
C:\ProgramData\Anaconda3\lib\site-packages\rdflib\plugins\serializers\rdfxml.py:280: UserWarning: Assertions on rdflib.term.BNode('N8c33a0fa4b494f7389e51ba6e8ec173c') other than RDF.first and RDF.rest are ignored ... including RDF.List
  self.predicate(predicate, object, depth + 1)
C:\ProgramData\Anaconda3\lib\site-packages\rdflib\plugins\serializers\rdfxml.py:280: UserWarning: Assertions on rdflib.term.BNode('Nbc2168bcbd38437f93770d30756772cb') other than RDF.first and RDF.rest are ignored ... including RDF.List
  self.predicate(predicate, object, depth + 1)
C:\ProgramData\Anaconda3\lib\site-packages\rdflib\plugins\serializers\rdfxml.py:280: UserWarning: Assertions on rdflib.term.BNode('N86edde0efee74062be5

OWL file saved at C:/USERS/User/Downloads/pop.owl
